# Distribuição de gabinetes

Considere que temos 3 gabinetes e queremos distribuir 4 pessoas (Ana=1, Nuno=2, Pedro=3 e Maria=4) por esses gabinetes.

Considere ainda que foram estipuladas as seguintes regras de ocupação dos gabinetes:

1. Cada pessoa ocupa um único gabinete.
2. O Nuno e o Pedro não podem partilhar gabinete.
3. Se a Ana ficar sozinha num gabinete, então o Pedro também terá que ficar sozinho num gabinete.
4. Cada gabinete só pode acomodar, no máximo, 2 pessoas.

Pretende-se que escreva um programa Python que, usando o Z3 como SAT solver, faça a distribuição das pessoas pelos gabinetes.

Começe por instalar o Z3.


In [1]:
!pip install z3-solver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 MB 17.1 MB/s eta 0:00:00


Para resolver o problema em Lógica Proposicional temos que

1. Declarar um conjunto de variáveis Boolenas
$x_{p,g}$ com a seguinte semântica:

  > $x_{p,g}$ é verdade  *sse*   a pessoa  $p$ ocupa o gabinete $g$

2. De seguida, teremos que modelar cada uma das restrições, acrescentando as fórmulas lógicas correspondentes.

3. Finalmente testamos se o conjunto de restrições é satisfazível e extraimos a solução calculada.  


Não tem solução.


In [17]:
from z3 import *

pessoas = ["Ana","Nuno","Pedro","Maria"]
gabs = [1,2,3]
x = {}
for p in pessoas:
    x[p] = {}
    for g in gabs:
        x[p][g] = Bool("%s,%d" % (p,g))

s = Solver()

# Cada pessoa ocupa pelo menos um gabinete.
for p in pessoas:
  s.add(Or([x[p][g] for g in gabs]))

# Cada pessoa ocupa no máximo um gabinete.
for p in pessoas:
  s.add(Implies(x[p][1], And(Not(x[p][2]),Not(x[p][3])) ))
  s.add(Implies(x[p][2],Not(x[p][3])))


# O Nuno e o Pedro não podem partilhar gabinete.
for g in gabs:
  s.add(Implies(x["Pedro"][g], Not(x["Nuno"][g])))

# Se a Ana ficar sozinha num gabinete, então o Pedro também terá
# que ficar sozinho num gabinete.
anaSo = Or([ And(x["Ana"][g],Not(x["Pedro"][g]),
                 Not(x["Maria"][g]),Not(x["Nuno"][g]))  for g in gabs])
pedroSo = Or([ And(x["Pedro"][g],Not(x["Ana"][g]),
                 Not(x["Maria"][g]),Not(x["Nuno"][g]))  for g in gabs])

s.add(Implies(anaSo,pedroSo))


# Cada gabinete só pode acomodar, no máximo, 2 pessoas.
comb = [("Ana","Nuno","Pedro","Maria"),("Ana","Pedro","Nuno","Maria"),
        ("Ana","Maria","Nuno","Pedro"),("Nuno","Pedro","Ana","Maria"),
        ("Nuno","Maria","Ana","Pedro"),("Pedro","Maria","Nuno","Ana")]
for g in gabs:
  s.add(And([Implies(And(x[p1][g],x[p2][g]), And(Not(x[p3][g]),Not(x[p4][g])))
                for (p1,p2,p3,p4) in comb]))



if s.check() == sat:
    m = s.model()
    for p in pessoas:
        for g in gabs:
            if is_true(m[x[p][g]]):
                print("%s fica no gabinete %s" % (p,g))
else:
  print("Não tem solução.")

Ana fica no gabinete 1
Nuno fica no gabinete 1
Pedro fica no gabinete 2
Maria fica no gabinete 3


Será que há várias alternativas para distribuir os gabinetes seguindo estas regras?

Faça as alterações necessárias ao programa de modo a saber todas as distribuições possíveis.

In [ ]:
s.push()
i=0

while s.check() == sat:
    i+=1
    m = s.model()
    f=[]
    for p in pessoas:
        for g in gabs:
            if is_true(m[x[p][g]]):
                print("%s fica no gabinete %s" % (p,g))
                f.append(Not(x[p][g]))
            else:
                f.append(x[p][g])
    s.add(Or(f))
    print()
else:
  print("Número de soluções: ", i)

s.pop()

Use agora o SAT solver para testar a veracidade se cada uma das seguintes afirmações:

1. Se a Maria ocupar o gabinete um, então ela ficará sozinha nesse gabinete.
2. Se a Ana e o Nuno ficarem no mesmo gabinete, então a Maria e o Pedro terão que partilhar também um outro gabinete.

A que conclusões chegou?


In [18]:
s.push()

# Se a Maria ocupar o gabinete um, então ela ficará sozinha nesse gabinete.
form1 = Implies(x["Maria"][1],
                And([ Not(x[p][1]) for p in pessoas if p!="Maria"]))

s.add(Not(form1))

print("A frase: \"Se a Maria ocupar o gabinete um, então ela ficará sozinha nesse gabinete\"")
if s.check() == unsat:
  print("é verdadeira.")
else:
  print("é falsa.")

s.pop()

A frase: "Se a Maria ocupar o gabinete um, então ela ficará sozinha nesse gabinete"
é falsa.


In [19]:
s.push()

# Se a Ana e o Nuno ficarem no mesmo gabinete, então a Maria e o Pedro
# terão que partilhar também um outro gabinete.
form2 = And([ Implies(And(x["Ana"][g],x["Nuno"][g]),
                     Or([ And(x["Maria"][a],x["Pedro"][a]) for a in gabs if a!=g]))
                for g in gabs ])

s.add(Not(form2))

print("A frase: \"Se a Ana e o Nuno ficarem no mesmo gabinete, então a Maria e o Pedro terão que partilhar também um outro gabinete.\"")
if s.check() == unsat:
  print("é verdadeira.")
else:
  print("é falsa.")

s.pop()

A frase: "Se a Ana e o Nuno ficarem no mesmo gabinete, então a Maria e o Pedro terão que partilhar também um outro gabinete."
é falsa.
